In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
class ForwardNet(nn.Module):
    def __init__(self):
        super(ForwardNet,self).__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(num_inputs,num_hiddens)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(num_hiddens,num_outputs)
    def forward(self,x):
        y = self.flatten(x)
        y = self.linear1(y)
        y = self.relu(y)
        y = self.linear2(y)
        return y